In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_parquet("./output/goldstandard_normal.parquet")

In [3]:
df.head()

,chain__,iter__,draw__,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,...,y[11],y[12],y[13],y[14],y[15],y[16],y[17],y[18],y[19],y[20]
0,1.0,1.0,1.0,-5.285098,0.805245,0.423067,3.0,7.0,0.0,20.038871,...,18.635339,18.850188,19.385812,19.376129,20.285174,18.535371,19.516910,19.856196,19.388709,19.708114
1,1.0,2.0,2.0,-8.271163,0.746428,0.423067,3.0,7.0,0.0,20.910310,...,18.606849,18.522111,19.496883,19.671718,19.762855,19.368333,20.154818,20.007807,18.974470,20.011775
2,1.0,3.0,3.0,-7.095761,0.976887,0.423067,3.0,7.0,0.0,16.321459,...,19.577178,19.773780,19.866859,19.050858,20.374573,20.130264,20.183197,19.023494,21.073913,17.891552
3,1.0,4.0,4.0,-12.320391,0.846098,0.423067,3.0,7.0,0.0,20.537464,...,19.177243,18.868033,20.211421,20.510142,18.491485,19.973557,19.459869,19.239501,21.725287,19.746145
4,1.0,5.0,5.0,-12.388545,0.999789,0.423067,3.0,15.0,0.0,21.469493,...,20.920675,21.221380,19.660246,19.686102,21.462170,20.142611,20.820015,20.617539,18.212552,20.129774


In [6]:
draws = df.iloc[:, 10:]

In [11]:
np.mean(draws, axis = 0), np.std(draws, axis = 0)

(y[1]     19.995345
 y[2]     20.003638
 y[3]     20.003808
 y[4]     20.000594
 y[5]     19.995056
 y[6]     19.997544
 y[7]     19.997712
 y[8]     20.003242
 y[9]     19.998672
 y[10]    19.998334
 y[11]    19.993383
 y[12]    20.000747
 y[13]    20.000575
 y[14]    19.990829
 y[15]    20.000257
 y[16]    20.004447
 y[17]    20.003135
 y[18]    19.998857
 y[19]    19.996749
 y[20]    20.010104
 dtype: float64,
 y[1]     1.007419
 y[2]     1.002960
 y[3]     1.006658
 y[4]     1.007747
 y[5]     1.006764
 y[6]     1.007747
 y[7]     0.998182
 y[8]     1.005044
 y[9]     1.010517
 y[10]    1.008375
 y[11]    1.001737
 y[12]    1.006420
 y[13]    1.007463
 y[14]    1.013847
 y[15]    1.002474
 y[16]    1.006410
 y[17]    1.009203
 y[18]    1.010335
 y[19]    1.007747
 y[20]    1.007354
 dtype: float64)